In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import os
import pathlib
import sys
import pandas as pd
sys.path.append('../dataset/')
sys.path.append('../models/')
from PIL import Image
from unet import unet_2D, UNet
import torch
from catalyst import dl, metrics, utils;
import albumentations as A
from dataset import AsbestosDataSet
from torch.utils.data import DataLoader
from torch import nn
import datetime 

In [2]:
image_dir = '../../task_asbestos_stone_lab_common_camera-2021_12_10_13_12_14-mots png 1.0/images/asbestos/stones/lab_common_camera/'
mask_dir  = '../../task_asbestos_stone_lab_common_camera-2021_12_10_13_12_14-mots png 1.0/SegmentationAsbest'

transform = A.Compose([A.Resize(1152, 1728), A.RandomCrop(1024,1024), A.Resize(512,512) ,A.RandomRotate90()])
dataset = AsbestosDataSet(image_dir, mask_dir,transform)
validation_set = AsbestosDataSet('../../task_asbestos_stone_lab_common_camera-2021_12_10_13_12_14-mots png 1.0/validation/images',
                           '../../task_asbestos_stone_lab_common_camera-2021_12_10_13_12_14-mots png 1.0/validation/masks',
                           A.Compose([A.Resize(1152, 1728), A.RandomCrop(1024,1024), A.Resize(512,512)]))

data_loader = DataLoader(dataset, num_workers=1, batch_size = 2,)
validation_data_loader = DataLoader(validation_set, num_workers=1, batch_size = 2,)

loaders = {"train":data_loader, "valid":validation_data_loader}


Number of images and masks are different


In [3]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()
    def forward(self, inputs, targets, smooth=1e-5):
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        return 1 - dice

In [4]:
import time

class CustomRunner(dl.Runner):
    def predict_batch(self, batch):
        return self.model(batch['image'].to(self.device))
    def on_loader_start(self, runner):
        super().on_loader_start(runner)
        self.meters = { key: metrics.AdditiveValueMetric(compute_on_call=True)
                        for key in ["loss", "iou"]}
        
    def handle_batch(self, batch):
        image, mask, name = batch.values()
        #----------
        time.sleep(0.12)
        image = image.unsqueeze(1)
        mask  = mask.unsqueeze(1)
#         mask  = torch.tensor(mask, dtype = torch.long)
        #------------
        predict = self.model(image)#batch size
#         print(image.shape, image.type(), mask.shape, mask.type())
        loss = self.criterion(predict, mask)
        #Создаем обьект
        iou_metric = metrics.IOUMetric()
        iou = iou_metric.update_key_value(predict, mask)['iou']
        self.batch_metrics.update(
            {"loss": loss, "iou": iou}
        )
        for key in ["loss", "iou"]:
            self.meters[key].update( self.batch_metrics[key].item(), self.batch_size)
        iou_metric.reset()
        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
    def on_loader_end(self, runner):
        for key in ["loss", "iou"]:
            self.loader_metrics[key] = self.meters[key].compute()[0]
        super().on_loader_end(runner)   

In [5]:
unet = UNet(n_channels=1, n_classes = 1 )
# unet = unet_2D(feature_scale=1, n_classes = 1, 
#                in_channels=1, is_deconv = True,
#                is_debug=False, )

In [ ]:
criterion = DiceLoss()
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(unet.parameters())
runner = CustomRunner()
runner.train(
    model=unet,
    criterion = criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=500, 
    logdir = 'unet_lab_log_{}'.format(str(datetime.datetime.now())),
    verbose=True)

/home/office/kirilman/miniconda3/lib/python3.8/site-packages/catalyst/core/runner.py:623: UserWarning: No ``ICriterionCallback/CriterionCallback`` were found while runner.criterion is not None.Do you compute the loss during ``runner.handle_batch``?
  warnings.warn(
/home/office/kirilman/miniconda3/lib/python3.8/site-packages/catalyst/core/runner.py:629: UserWarning: No ``IOptimizerCallback/OptimizerCallback`` were found while runner.optimizer is not None.Do run backward pass during ``runner.handle_batch``?
  warnings.warn(


/home/office/kirilman/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



train (1/500) iou: 0.05897066038515833 | loss: 0.8891631298595004



valid (1/500) iou: 0.05021339245140553 | loss: 0.9047295451164246
* Epoch (1/500) 



train (2/500) iou: 0.0761429982052909 | loss: 0.8596892522441015


...


valid (61/500) iou: 0.2550720810890198 | loss: 0.5959863781929016
* Epoch (61/500) 



train (62/500) iou: 0.36476179709037143 | loss: 0.4729916950066884



valid (62/500) iou: 0.27594802975654603 | loss: 0.5792726278305054
* Epoch (62/500) 



train (63/500) iou: 0.34551861633857095 | loss: 0.4897909263769785



valid (63/500) iou: 0.2289018452167511 | loss: 0.6330934166908264
* Epoch (63/500) 
